# Mechanistic Interpretability of Jailbreak Prompts

In this notebook, we'll explore how language models respond to various jailbreak prompts of different strengths compared to a "clean" dataset of prompts. The goal is to:
- Load and store these prompts within Colab (mounted to Google Drive).
- Potentially run inference on a model to analyze where "jailbreak" behavior arises.
- Lay groundwork for mechanistic interpretability (e.g., analyzing specific model attention heads, hidden states, or other internal mechanisms).

We will start small, with:
1. Data loading (Google Drive).
2. Basic data inspection and prompt categorization.
3. Setting up inference code for a small language model (e.g., a Hugging Face model) as a test bed.

In [1]:
# Check Python version (optional):
import sys
print("Python version:", sys.version)

# Get installations
!pip install --quiet torch numpy matplotlib scikit-learn pandas
!pip install --quiet huggingface_hub transformers

import os
import torch
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

# If you want to check GPU usage:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

Python version: 3.11.11 (main, Dec  4 2024, 08:55:07) [GCC 11.4.0]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 61.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 51.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 36.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 57.9 MB/s eta 0:00:00
Using device: cuda


In [1]:
from google.colab import drive
drive.mount('/content/drive')

# After running this cell, follow the link to grant Colab access to your Google Drive.

In [2]:
!git clone https://github.com/IdaCy/jailbreak-alert-research.git

Cloning into 'jailbreak-alert-research'...
remote: Enumerating objects: 184, done.
remote: Counting objects: 100% (184/184), done.
remote: Compressing objects: 100% (119/119), done.
remote: Total 184 (delta 68), reused 152 (delta 38), pack-reused 0 (from 0)
Receiving objects: 100% (184/184), 190.68 KiB | 5.61 MiB/s, done.
Resolving deltas: 100% (68/68), done.


In [3]:
%ls

jailbreak-alert-research/  sample_data/


In [4]:
%cd jailbreak-alert-research

/content/jailbreak-alert-research


In [5]:
!git pull

Already up to date.


In [ ]:
!pip install huggingface_hub --quiet

from huggingface_hub import notebook_login

# This will prompt you in Colab to enter your HF token or log in directly
notebook_login()

## Run inference on the **general jailbreak** prompts:

In [ ]:
PROMPT_FILE = "data/ReNeLLM/jailbreaks/jb400.csv"
OUTPUT_DIR = "output/extractions/jailbreak"

MODEL_NAME = "google/gemma-2-9b"
HF_TOKEN = None
BATCH_SIZE = 2
NUM_SAMPLES = 10

%run -i scripts/gemma2b/inference.py

## Run inference on any **other** prompts:

In [ ]:
PROMPT_FILE = "data/ReNeLLM/neutral/neutr400.csv"

# ATTENTION: don't forget creating the folder first
OUTPUT_DIR = "output/extractions/neutral"

MODEL_NAME = "google/gemma-2-9b"
BATCH_SIZE = 2

%run -i scripts/gemma2b/inference.py

## Run inference on any **other** prompts?

In [ ]:
PROMPT_FILE = "data/ReNeLLM/jailbreak_stronger/jb400.csv"

# ATTENTION: don't forget creating the folder first
OUTPUT_DIR = "output/extractions/jailbreak_stronger"

MODEL_NAME = "google/gemma-2-9b"
BATCH_SIZE = 2

%run -i scripts/gemma2b/inference.py

# Get the differences & PCA of any two (or more!)

In [ ]:
JB_DIR = "output/extractions/jailbreak"
NEUTRAL_DIR = "output/extractions/neutral"
DIFF_DIR = "output/diff"

%run -i scripts/analyses/1_compute_diff.py

In [ ]:
DIFF_DIR = "output/differences"
OUTPUT_DIR = "output/PCA"

%run -i scripts/analyses/2_run_pca.py

In [ ]:
results_pt = "output/PCA/layer_pca_results.pt"
PC1_FILE = "output/PCA/layer_pc1_vectors.pt"

%run -i scripts/analyses/3_pca_check.py